# Tutorial
***
This tutorial covers most of the features available in the Python source code. I hope that this serves as a helpful "get started" resource.

## Projective and Injective Modules
***

Firstly, we'll import the required files.

In [1]:
from relations import *
from linear_quivers import *
from linear_module import *

One of the first things that we want to do is create a linear quiver. To do so, we must know two things: the number of vertices in the quiver and quiver's relations. We will initialise the number of vertices to 5 and the relations to be the empty list, representing no relations.

In [2]:
n = 5 # Number of vertices in the quiver
relations = [] # The quiver's relations

We can now create a quiver using these two variables.

In [3]:
lq = linear_quiver(n, relations) # Creating the linear quiver

When a ``linear_quiver`` object is created, the private method ``__calculate_jectives`` gets called in the constructor. This method calculates the quiver's projective and injective modules, and stores them in the class variable ``self.jectives``. 

We define a **"jective"** to be the pair ``(projective_module, injective_module)``. There are four getter methods related to these jectives:
* ``get_jectives()`` returns ``self.jectives`` (a list containing all of the jectives)
* ``get_nth_jective(n)`` returns the $n$<sup>th</sup> jective from ``self.jectives``
* ``get_nth_projective(n)`` returns the $n$<sup>th</sup> projective module
* ``get_nth_injective(n)`` returns the $n$<sup>th</sup> injective module

We'll now use these methods on our quiver ``lq``.

In [4]:
jectives = lq.get_jectives() # Obtaining the "jectives"
jectives

[[[1, 1], [5, 1]],
 [[2, 1], [5, 2]],
 [[3, 1], [5, 3]],
 [[4, 1], [5, 4]],
 [[5, 1], [5, 5]]]

**Note:** Here, we are making use of shorthand notation for the injective and projective modules. We use ``[a,b]``, with $a \geq b$, for the projective and injective modules, which is shorthand for the module with composition factors``[a, a-1, ..., b+1, b]``.

In [5]:
third_jective = lq.get_nth_jective(3) # Obtaining the third jective
third_jective

[[3, 1], [5, 3]]

Alternatively, we can obtain the third projective and injective module separately.

In [6]:
third_injective = lq.get_nth_injective(3) # Obtaining the third injective module
third_projective = lq.get_nth_projective(3) # Obtaining the third projective module
[third_projective, third_injective]

[[3, 1], [5, 3]]

## Projective Resolutions
***

Given a module $M$, we can compute its projective resolution by calling the aptly named ``projective_resolution_of_module`` class method.

In [7]:
M = [3,3] # Our module of choice
pr_M = lq.projective_resolution_of_module(M)
pr_M

[3, 2]

One useful thing that we can do is compute the projective resolution of the injective modules for ``lq``. This is done easily enough using the ``projective_resolution_of_injectives`` class method.

In [8]:
proj_res_inj = lq.projective_resolution_of_injectives() # Obtaining the projective resolution]
proj_res_inj

[[5], [5, 1], [5, 2], [5, 3], [5, 4]]

After computing this, we can find the matrix of this list of projective resolutions using the static class method ``matrix_of_proj_res``, which takes a list of $n$ projective modules as its parameter.

In [9]:
proj_res_mat = lq.matrix_of_proj_res(proj_res_inj) # Obtaining the matrix of the projective resolution
proj_res_mat

array([[ 0, -1,  0,  0,  0],
       [ 0,  0, -1,  0,  0],
       [ 0,  0,  0, -1,  0],
       [ 0,  0,  0,  0, -1],
       [ 1,  1,  1,  1,  1]])

With this matrix, we can then check whether the algebra ``lq`` appears to be fractional Calabi-Yau (fCY) on the Grothendieck group by checking whether ``proj_res_mat`` has finite order. This can be done using the static class method ``is_fcy_gg``, which takes (in this order) a projective resolution matrix, a maximum integer power to raise the matrix to, and an optional boolean value for verbose output (defaulted to ``False``) as parameters.

In [10]:
fcy_output = linear_quiver.is_fcy_gg(proj_res_mat, 50) 
fcy_output

(True, 6, '+')

The ``fcy_output`` tuple tells us that ``lq`` appears to be fCY on the Grothendieck group (indicated by `True`) and that we must raise ``proj_res_mat`` to the power of 6 for it to be the identity matrix (indicated by the `'+'`). If the third tuple element was `'-'`, this would indicate that ``proj_res_mat`` raised to the sixth power was negative the identity matrix.

We can also get verbose output, which prints the matrix at each stage of multiplication.

In [11]:
fcy_output = linear_quiver.is_fcy_gg(proj_res_mat, 50, verbose = True) 
fcy_output

Power:  2
[[ 0  0  1  0  0]
 [ 0  0  0  1  0]
 [ 0  0  0  0  1]
 [-1 -1 -1 -1 -1]
 [ 1  0  0  0  0]]

Power:  3
[[ 0  0  0 -1  0]
 [ 0  0  0  0 -1]
 [ 1  1  1  1  1]
 [-1  0  0  0  0]
 [ 0 -1  0  0  0]]

Power:  4
[[ 0  0  0  0  1]
 [-1 -1 -1 -1 -1]
 [ 1  0  0  0  0]
 [ 0  1  0  0  0]
 [ 0  0  1  0  0]]

Power:  5
[[ 1  1  1  1  1]
 [-1  0  0  0  0]
 [ 0 -1  0  0  0]
 [ 0  0 -1  0  0]
 [ 0  0  0 -1  0]]

Power:  6
[[1 0 0 0 0]
 [0 1 0 0 0]
 [0 0 1 0 0]
 [0 0 0 1 0]
 [0 0 0 0 1]]



(True, 6, '+')

Since we know that the power must be 6, if we choose to test to a power lower than 6, then we should expect to get a different output.

In [12]:
fcy_output_alt = linear_quiver.is_fcy_gg(proj_res_mat, 5)
fcy_output_alt

(False, 5, 'N/A')

Here, the ``False`` indicates that we never reached plus/minus the identity matrix, the ``5`` represents the maximum power that we raised the matrix to, and ``N/A`` is in place of ``+`` or ``-``.

## Serre Functors

Moving away from matrices and back to projective resolutions, we will begin by showing how to apply the Serre functor to modules. We'll do so by applying the Serre functor to the module ``[3]``.

In [13]:
module = [3] # Our module of choice
serre_third = lq.serre_functor([module]) # Applying the Serre functor to the module [3]
serre_third

[[5], [2]]

**Note:** ``serre_functor`` expects to be passed a list of modules, so we had to pass ``[module]`` to the ``serre_functor`` method instead of ``module``.

What's useful about this method, is that we can apply it to the output.

In [14]:
serre_third = lq.serre_functor(serre_third) # Applying the Serre functor to the third projective twice
serre_third

[[], [4], [1]]

Let's go slightly further and use a ``for`` loop to apply the Serre functor 10 times to the module ``[1]``.

In [15]:
mod = [1] # Obtaining the fourth projective module
serre_out = lq.serre_functor([mod]) # Applying the Serre functor once

print(0, [[1]])
print(1, serre_out)

for i in range(9):
    serre_out = lq.serre_functor(serre_out)
    print(i+2, serre_out)


0 [[1]]
1 [[5]]
2 [[5], [4]]
3 [[], [4], [3]]
4 [[], [], [3], [2]]
5 [[], [], [], [2], [1]]
6 [[], [], [], [], [1]]
7 [[], [], [], [], [5]]
8 [[], [], [], [], [5], [4]]
9 [[], [], [], [], [], [4], [3]]
10 [[], [], [], [], [], [], [3], [2]]


Finally, we can use the class method ``fcy_dim`` to compute the Calabi-Yau dimension of ``lq``. This method takes a single integer parameter referring to the maximum number of times to apply the Serre functor.

In [16]:
dim = lq.fcy_dim(40) 
dim

(4, 6)

The output of ``fcy_dim`` is a tuple $(r,s)$ representing that $\$^s \cong \Sigma^r$. In our case, we obtained a Calabi-Yau dimension of the form ``(n-1, n+1)``, with $n=5$ in our quiver `lq`, which is exactly the resulted expected for a quiver with no relations.

Similar to ``is_fcy_gg``, we can also get verbose output detailing the Serre functor's output at each step.

In [17]:
dim = lq.fcy_dim(40, verbose = True) 
dim

0 --> [[[1]], [[2]], [[3]], [[4]], [[5]]]
1 --> [[[5]], [[5], [1]], [[5], [2]], [[5], [3]], [[5], [4]]]
2 --> [[[5], [4]], [[], [4]], [[], [4], [1]], [[], [4], [2]], [[], [4], [3]]]
3 --> [[[], [4], [3]], [[], [5], [3]], [[], [], [3]], [[], [], [3], [1]], [[], [], [3], [2]]]
4 --> [[[], [], [3], [2]], [[], [], [4], [2]], [[], [], [5], [2]], [[], [], [], [2]], [[], [], [], [2], [1]]]
5 --> [[[], [], [], [2], [1]], [[], [], [], [3], [1]], [[], [], [], [4], [1]], [[], [], [], [5], [1]], [[], [], [], [], [1]]]
FINAL --> [[[], [], [], [], [1]], [[], [], [], [], [2]], [[], [], [], [], [3]], [[], [], [], [], [4]], [[], [], [], [], [5]]]


(4, 6)

Since our dimension is ``(4,6)``, we know that must apply the Serre functor 4 times. Since $40 \geq 4$, we were able to calculate the dimension. However, if we passed ``3`` instead of ``40`` into the ``fcy_dim`` function, we would get a different output.

In [18]:
dim = lq.fcy_dim(3)
dim

'max_iter reached: 3'

This output tells us that for every $1 \leq i \leq 3$, we have that $\$^i \not\cong \Sigma^r$ for some $r$. After applying the Serre functor 3 times, the algorithm stops and returns this output.

## Generating Relations
***

The file ``relations.py`` has a single function: ``generate_length_k_relations``. This takes two parameters: ``n`` (the number of vertices in the quiver) and ``k`` (the length of the relations). Utilising this, we can write a simple script to perform an fCY test for a quiver against all length ``k`` relations.

In [19]:
relations = length_k_relations(5, 3) # All length 3 three for a quiver with 5 vertices
relations

[((1, 4),), ((2, 5),), ((1, 4), (2, 5))]

In [20]:
for rel in relations:
    lq = linear_quiver(5, rel)
    pr = lq.projective_resolution_of_injectives()
    mat_pr = lq.matrix_of_proj_res(pr)
    print(linear_quiver.is_fcy_gg(mat_pr, 50))

(True, 8, '+')
(True, 8, '+')
(True, 8, '+')


We can also do a similar thing for Serre resolutions.

In [21]:
for rel in relations:
    lq = linear_quiver(5, rel)
    print(lq.fcy_dim(50))

(4, 8)
(4, 8)
(4, 8)


***
**END OF TUTORIAL**